In [ ]:
import cracknuts as cn

In [ ]:
cracker = cn.new_cracker('127.0.0.1')
# cracker

In [ ]:
import time
def do(cracker):
    time.sleep(0.1)
    return b'123123'

acq = cn.new_acquisition(cracker, do=do)
# acq

In [ ]:
# cn.monitor_panel(acq)

In [ ]:
cn.panel(acq)

In [ ]:
import logging

# 配置日志记录
logger = logging.getLogger('jupyter_log')
logger.setLevel(logging.DEBUG)  # 设置日志级别

# 创建指向标准错误的控制台处理器
console_handler = logging.StreamHandler()  # 默认使用 sys.stderr
console_handler.setLevel(logging.DEBUG)

# 设置日志格式
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# 清除已有的处理器，防止重复日志
if logger.hasHandlers():
    logger.handlers.clear()

# 将处理器添加到 logger
logger.addHandler(console_handler)

# 打印日志
logger.info("This message will appear in the terminal, not in the notebook cell.")
logger.warning("This is a warning.")
logger.error("This is an error.")



In [ ]:
cracker.osc_set_sample_phase(60)